In [1]:
!python3 -m pip install --upgrade langchain deeplake openai tiktoken

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/5c/c2/66a16f85f5fc275ba3436a7862d7d89f736682687e2c93359e8ab6541dae/langchain-0.0.283-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.1/541.1 kB 1.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/ae/59/911d6e5f1d7514d79c527067643376cddcf4cb8d1728e599b3b03ab51c69/openai-0.28.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.4/761.4 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for PyYAML>=5.3 from https://files.pythonhosted.org/packages/5b/07/10033a403b23405a8fc48975444463d3d10a5c2736b7eb2550b07b367429/PyYAML-6.0.1-cp310-cp310-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for SQLAlchemy<3,>=1.4 from

In [1]:
 
import os
 
import getpass
 
 
 
from langchain.embeddings.openai import OpenAIEmbeddings
 
from langchain.vectorstores import DeepLake
 
 
 
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
 
os.environ['ACTIVELOOP_TOKEN'] = getpass.getpass('Activeloop Token:')
 

In [3]:
embeddings = OpenAIEmbeddings(disallowed_special=())

In [7]:
 
!git clone https://github.com/lets-blade/blade.git

Cloning into 'blade'...
remote: Enumerating objects: 11679, done.
remote: Counting objects: 100% (2449/2449), done.
remote: Compressing objects: 100% (914/914), done.
remote: Total 11679 (delta 1036), reused 2302 (delta 968), pack-reused 9230
Receiving objects: 100% (11679/11679), 3.35 MiB | 372.00 KiB/s, done.
Resolving deltas: 100% (5516/5516), done.


In [4]:
 
import os
 
from langchain.document_loaders import TextLoader
 
 
 
root_dir = './blade/blade-core'
 
docs = []
 
for dirpath, dirnames, filenames in os.walk(root_dir):
 
    for file in filenames:
 
        try: 
 
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
 
            docs.extend(loader.load_and_split())
 
        except Exception as e: 
 
            pass

In [5]:
 
from langchain.text_splitter import CharacterTextSplitter
 
 
 
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
 
texts = text_splitter.split_documents(docs)

Created a chunk of size 1739, which is longer than the specified 1000
Created a chunk of size 3624, which is longer than the specified 1000
Created a chunk of size 1207, which is longer than the specified 1000
Created a chunk of size 1316, which is longer than the specified 1000
Created a chunk of size 1145, which is longer than the specified 1000
Created a chunk of size 1139, which is longer than the specified 1000
Created a chunk of size 1065, which is longer than the specified 1000
Created a chunk of size 1900, which is longer than the specified 1000
Created a chunk of size 2162, which is longer than the specified 1000
Created a chunk of size 1043, which is longer than the specified 1000
Created a chunk of size 1486, which is longer than the specified 1000
Created a chunk of size 1175, which is longer than the specified 1000
Created a chunk of size 1147, which is longer than the specified 1000
Created a chunk of size 1077, which is longer than the specified 1000
Created a chunk of s

In [9]:
import time

username = "geekywizkid" 

db = DeepLake(
    dataset_path=f"hub://{username}/blade_core_1",
    embedding_function=embeddings,
    public=True
)

db.add_documents([text]) # 每次只上传一个文档
time.sleep(20) # 等待20秒,实现每分钟3次请求的频率，如果是付费号请忽略

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in hub://geekywizkid/blade_core_1 already exists, loading from the storage


续写用这个

In [15]:
import time

username = "geekywizkid" 

db = DeepLake(dataset_path="hub://geekywizkid/blade_core_1", read_only=False, embedding_function=embeddings)

text_count = 0

for text in texts:
    if ++text_count > 129: #前面的已经成功上传了128个文档
        db.add_documents([text]) # 每次只上传一个文档
        time.sleep(20) # 等待20秒,实现每分钟3次请求的频率

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in hub://geekywizkid/blade_core_1 already exists, loading from the storage


问答

In [10]:
 
db = DeepLake(dataset_path="hub://geekywizkid/blade_core_1", read_only=True, embedding_function=embeddings)

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in hub://geekywizkid/blade_core_1 already exists, loading from the storage


In [11]:
retriever = db.as_retriever()
 
retriever.search_kwargs['distance_metric'] = 'cos'
 
retriever.search_kwargs['fetch_k'] = 100
 
retriever.search_kwargs['maximal_marginal_relevance'] = True
 
retriever.search_kwargs['k'] = 10
 

In [12]:
from langchain.chat_models import ChatOpenAI
 
from langchain.chains import ConversationalRetrievalChain
 
 
 
model = ChatOpenAI(model_name='gpt-3.5-turbo') # switch to 'gpt-4'
 
qa = ConversationalRetrievalChain.from_llm(model,retriever=retriever)

In [16]:
questions = [
 
    "Who is the founder of Blade?",
    "How to install Blade?",
    "explain BladeCache.kt",
 
] 
 
chat_history = []
 
 
 
for question in questions:  
 
    result = qa({"question": question, "chat_history": chat_history})
 
    chat_history.append((question, result['answer']))
 
    print(f"-> \*\*Question\*\*: {question} ")
 
    print(f"\*\*Answer\*\*: {result['answer']} ")

-> \*\*Question\*\*: Who is the founder of Blade? 
\*\*Answer\*\*: The founder of Blade is biezhi. 


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-pWkCQMCDQn5J5mkjAHItKYip on requests per day. Limit: 200 / day. Please try again in 7m12s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-pWkCQMCDQn5J5mkjAHItKYip on requests per day. Limit: 200 / day. Please try again in 7m12s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.ope

KeyboardInterrupt: 